## This script generates a video from a sequence of pictures.

In [1]:
import cv2
import os, sys
from tqdm import tqdm
from videolib import initVideoWriter, imageHiveOverview
sys.path.append("..")
from ForegroundRemoval.PercentileFilter.percentile import percentile_filter_single
import pandas as pd
pd.set_option('display.width', 1000)
sys.path.append("..")
from libimage import fetchImagesPaths
from dask.distributed import Client
from dask import compute
# Start a client using all CPUs of this machine
client = Client()
client

/opt/anaconda3/envs/imaging/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 58589 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:58589/status,
Dashboard: http://127.0.0.1:58589/status,Workers: 4
Total threads: 12,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:58590,Workers: 0
Dashboard: http://127.0.0.1:58589/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:58601,Total threads: 3
Dashboard: http://127.0.0.1:58602/status,Memory: 8.00 GiB
Nanny: tcp://127.0.0.1:58593,


## Configuration

In [2]:
# Path to the folder containing the pictures
rootpath = '/Users/cyrilmonette/Library/CloudStorage/SynologyDrive-data/25.07-25.10_aSensing_OH/Images/'
rootpath_img_period = 5 # Period between every image in the folder in minutes
hive_nb = 1

# The following are in CET time zone:
first_dt_str = "250731-150000"
last_dt_str = "251016-100000"

frame_drop = 1 # We keep 1 frame every frame_drop frames. Put one to keep all frames.
fps_video = 10 # Frames per second for the video

remove_bees = False # If True, we remove bees using a percentile filter

## Main code

In [3]:
first_dt = pd.to_datetime(first_dt_str, format="%y%m%d-%H%M%S").tz_localize('CET')
last_dt = pd.to_datetime(last_dt_str, format="%y%m%d-%H%M%S").tz_localize('CET')
assert first_dt < last_dt, "First date must be before last date"
print("First date: ", first_dt)
print("Last date: ", last_dt)

time_range = pd.date_range(start=first_dt, end=last_dt, freq=f"{rootpath_img_period*frame_drop}min")
time_range = time_range.to_list()
print("Length time range: ", len(time_range))
img_paths = fetchImagesPaths(rootpath, time_range, hive_nb, invalid_recovery_time=120,images_fill_limit=None)
print(img_paths.head())

# Find the first index of img_paths that does not contain Nones in any column
first_valid_index = img_paths.dropna(how='all', inplace=False).index[0]
example_frame = imageHiveOverview([cv2.imread(img_paths.loc[first_valid_index].iloc[j]) for j in range(4)], False,
                                  ["hive" + str(hive_nb) + "_rpi" + str(j+1) + "_" + img_paths.loc[first_valid_index].iloc[j] for j in range(4)])
name = "hive" + str(hive_nb) + "_" + first_dt.tz_convert('UTC').strftime("%y%m%d-%H%M%SZ") + "_" + last_dt.tz_convert('UTC').strftime("%y%m%d-%H%M%SZ")
if remove_bees:
    name += "_beesRemoved"
video = initVideoWriter(dest="outputVideos/", frame=example_frame,name=name, fps=fps_video, grayscale=False)
print("Writing video with dimensions: ", example_frame.shape)

# Iterate over img_paths DataFrame
for i, row in tqdm(img_paths.iterrows(), total=img_paths.shape[0]):
    imgs = []
    names = []
    validity = True 
    if 'valid' in row:
        validity = row['valid']
        row.drop('valid', inplace=True)

    for j in range(len(row)):
        if row.iloc[j] is not None:
            file_path = row.iloc[j]
            names.append(os.path.basename(file_path))  # Get the file name
            if remove_bees:
                _img, _ = percentile_filter_single(file_path,percentile=90, filter_length=20, frame_skip=5, annotate_names=False, verbose=False)
            else:
                _img = cv2.imread(file_path)
            imgs.append(_img)
        else:
            # If the file does not exist, append a black image
            names.append("No image available")
            # Append a black image of the same size as the example frame
            imgs.append(cv2.imread(img_paths.loc[first_valid_index].iloc[j]) * 0)

    if remove_bees:
        imgs = compute(*imgs)
        # Convert the images to rgb from grayscale
        imgs = [cv2.cvtColor(img, cv2.COLOR_GRAY2BGR) if len(img.shape) == 2 else img for img in imgs]
    assembled_img = imageHiveOverview(imgs, img_names=names, dt=i, valid=validity)
    video.write(assembled_img)

# Release the VideoWriter object
video.release()
print("Video written successfully.")

First date:  2025-07-31 15:00:00+02:00
Last date:  2025-10-16 10:00:00+02:00
Length time range:  22117
                                                                        h1r1                                               h1r2                                               h1r3                                               h1r4  valid
2025-07-31 15:00:00+02:00  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/cyrilmonette/Library/CloudStorage/Synol...   True
2025-07-31 15:05:00+02:00  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/cyrilmonette/Library/CloudStorage/Synol...   True
2025-07-31 15:10:00+02:00  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users

100%|██████████| 22117/22117 [1:35:07<00:00,  3.88it/s]  

Video written successfully.
